# VGG 16

In [2]:
import time
import numpy as np
import pandas as pd

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Conv2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

# Read data sets
y_train = pd.read_csv('Train/trainLbls.csv', header=None, names=['label'])['label']
y_validation = pd.read_csv('Validation/valLbls.csv', header=None, names=['label'])['label']
X_test = pd.read_csv('Test/testVectors.csv', header=None).transpose()

img_width, img_height = 256, 256
train_data_dir = "Train/TrainImages"
validation_data_dir = "Validation/ValidationImages"
test_data_dir = "Test/TestImages"

n_train_samples = len(y_train)
n_validation_samples = len(y_validation)
n_test_samples = X_test.shape[0]
n_labels = len(y_train.unique())

N_FROZEN_LAYERS = 5
BATCH_SIZE = 16
EPOCHS = 100
LEARNING_RATE = 0.0001
VERBOSE = 0
DATA_AUGMENTATION_FACTOR = 5

# Define checkpoint
checkpoint = ModelCheckpoint(
    'model-02-vgg16-epoch{epoch:02d}-valacc{val_acc:.2f}-valloss{val_loss:.2f}.hdf5', 
    monitor='val_acc', 
    save_best_only=False, 
    save_weights_only=False, 
    mode='auto', 
    period=1,
    verbose=VERBOSE
)

# Define early stopping
early = EarlyStopping(
    monitor='val_acc', 
    min_delta=0, 
    patience=10,
    mode='auto', 
    verbose=VERBOSE
)

# ImageDataGenerator generates batches of normalised image data i.e.
# a format that the images must be in to be read by the Keras model
train_batches = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=30
).flow_from_directory(
    train_data_dir,
    target_size = (img_height, img_width),
    batch_size = BATCH_SIZE,
    class_mode = "categorical")

validation_batches = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=30
).flow_from_directory(
    validation_data_dir,
    target_size = (img_height, img_width),
    batch_size = BATCH_SIZE,
    class_mode = "categorical")

Found 5830 images belonging to 29 classes.
Found 2298 images belonging to 29 classes.


In [13]:
n_test_samples

3460

In [15]:
vgg_model = applications.VGG19(weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3))

In [16]:
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [3]:
vgg_model = applications.VGG16(weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3))

In [4]:
# Convert the VGG model to Sequential model
model = Sequential(vgg_model.layers)

In [5]:
for layer in model.layers[:N_FROZEN_LAYERS]:
    layer.trainable = False

In [8]:
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dense(n_labels, activation='softmax'))

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [10]:
model.compile(
    optimizers.Adam(lr=LEARNING_RATE), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [ ]:
model.fit_generator(
    train_batches, 
    steps_per_epoch=int(n_train_samples/BATCH_SIZE) * DATA_AUGMENTATION_FACTOR,
    validation_data=validation_batches,
    validation_steps=int(n_validation_samples/BATCH_SIZE) * DATA_AUGMENTATION_FACTOR,
    epochs=EPOCHS,
    callbacks=[checkpoint, early],
    use_multiprocessing=True
    verbose=VERBOSE,
)